# Polygonising pixel edges <img align="right" src="../../Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with the `NCI`|`DEA Sandbox` environment only
* **Products used:** 
[s2a_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), 
[s2b_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule),
[ga_ls5t_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3),
[ga_ls7e_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
[ga_ls8c_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3)

## Background
The ability to get the pixel boundaries as a shapefile or other vector file type is very useful for remote sensing applications where the boundaries of the pixel are needed. 
This is useful for cases such as matching drone imagery with remotely sensed imagery. 


## Description
This notebook uses Digital Earth Australia to retrieve satellite data, creates a vector file from the pixel boundaries of the data, and exports to file.

1. First we load some data for a chosen time frame using the dea-notebooks `load_ard` function
2. Then we convert our raster data into a polygon per pixel
3. Then we export our pixel edges polygons as a vector file (shapefile)
4. This loop is repeated (once for Sentinel 2 data and once for Landsat data)

***

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Scripts` directory.

In [ ]:
%matplotlib inline
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import rasterio.features
from shapely.geometry import shape
import sys
import xarray as xr

sys.path.append("../Scripts")
from dea_datahandling import load_ard
from dea_plotting import rgb

from datacube.utils import masking
from datacube.utils.cog import write_cog
from datacube import Datacube
from datetime import datetime
from datacube import __version__

%load_ext autoreload
%autoreload 2

### Define useful functions
The function below is a helper function to help you make a nice filename for the output files

In [ ]:
def create_filename(ds, image_index, query, prefix=""):
    """create_filename is a handy function to turn your xarray dataset and image number into a nice output filename for your shapefile.
    ds : an xarray dataset
    image_index : an integer value between 0 and however many images you have
    prefix : a string : an optional name prefix to identify your area of interest"""
    assert ds
    image_date = str(ds.nbart_red[image_index].time.data)[:10]
    image_coords = f"{query['x'][0]}_{query['y'][0]}_{query['x'][1]}_{query['y'][1]}"
    if prefix:
        output_name = f"{str(prefix)}_{image_date}_{image_coords}"
    else:
        output_name = f"{image_date}_{image_coords}"
        print(output_name)
    return output_name

### Connect to the datacube

Connect to the datacube so we can access DEA data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [ ]:
dc = Datacube(app="Polygonise pixel edges")

### Analysis parameters
* `x` : a tuple of longitude/Easting boundaries defining the area of interest in the format (smaller, larger) e.g. `(153.40, 153.50)`
* `y` : a tuple of latitude/Northing boundaries defining the area of interest in the format (smaller, larger) e.g. `(-27.40, -27.50)`
* `time` : a tuple of time values in a pandas datetime format defining the time period of interest in the format (smaller, larger) e.g. `("2018-08-01", "2018-08-20")`
* `products` : A list of product names to load from the datacube e.g. `['ga_ls7e_ard_3', 'ga_ls8c_ard_3']`
* `measurements` : A list of band names to load from the satellite product e.g. `['nbart_red', 'nbart_green']`
* `resolution` : The spatial resolution of the loaded satellite data e.g. for Landsat, this is `(-30, 30)`
* `output_crs` : The coordinate reference system/map projection to load data into, e.g. `'EPSG:3577'` to load data in the Albers Equal Area projection
* `group_by` : How to group the data e.g. `"solar_day"`

### Set up a datacube query for our area of interest
Grab Sentinel 2 satellite data first

In [ ]:
# Create a query object
s2_query = {
    "x": (153.40, 153.50),
    "y": (-27.40, -27.50),
    "time": ("2018-08-01", "2018-08-20"),
    "products": ("s2a_ard_granule", "s2b_ard_granule"),
    "measurements": ["nbart_red", "nbart_green", "nbart_blue", "fmask"],
    "output_crs": "EPSG:3577",
    "resolution": (-10, 10),
    "group_by": "solar_day",
}

Load available data from Sentinel 2 satellites

In [ ]:
# Load sentinel2 data using load_ard. We're not using fmask to mask our pixel quality, as we're after the surrounds of the pixels not big white blobs.
s2_ds = load_ard(
    dc=dc,
    min_gooddata=0.95,  # only take scenes with less than 5% fmask activations
    mask_pixel_quality=False,
    **s2_query
)

### Plot up some data to take a look at it
Use the `rgb` function from `dea-plotting` module to look at our area of interest in true colour

In [ ]:
rgb(s2_ds, bands=["nbart_red", "nbart_green", "nbart_blue"], col="time")

### Choose an image index to use to polygonise the pixels
Change the number in the cell below to select a different image from the images in the previous cell; 0 is the first image.

In [ ]:
image_index = 0

### Segment our image into per-pixel-polygons
Use the red band to create a polygon for each pixel. 

In [ ]:
# Input array to segment and vectorise
input_array = s2_ds.nbart_red[image_index]
input_transform = s2_ds.affine  
input_crs = s2_ds.crs

# Create array with a unique value per cell
unique_pixels = np.arange(input_array.size).reshape(input_array.shape)

# Vectorise each unique feature in array
vectors = rasterio.features.shapes(
    source=unique_pixels.astype(np.int16), transform=input_transform
)

# Extract polygons and values from generator
vectors = list(vectors)
values = [value for polygon, value in vectors]
polygons = [shape(polygon) for polygon, value in vectors]

# Create a geopandas dataframe populated with the polygon shapes
s2_poly_gdf = gpd.GeoDataFrame(data={"id": values}, geometry=polygons, crs=input_crs)

### Investigate the results
Print and plot the first 5 entries in the polygon geodataframe 

In [ ]:
# print the first 5 rows in the geodataframe
s2_poly_gdf.head()

In [ ]:
# plot the first five polygons in the geodataframe, coloured by id
s2_poly_gdf.head().plot(column="id", edgecolor="black", figsize=(20, 20))
plt.show()

### Write our polygonised pixel edges for our area of interest out to file
Change the prefix in the cell below for a customised filename

In [ ]:
output_filename = create_filename(s2_ds, image_index, s2_query, prefix="NS_s2")

In [ ]:
# write our new segmented shapefile out to a file
s2_poly_gdf.to_file(output_filename + ".shp")

### Write the corresponding RGB geotiff image data out to file, for checking 

In [ ]:
# write the corresponding geotiff out to a file, for checking purposes
s2_rgb = s2_ds.isel(time=image_index).to_array()
write_cog(s2_rgb, output_filename + ".tif")

### Grab the landsat satellite data to reproduce our workflow 
Grab landsat satellite data first

In [ ]:
# Create a landsat query
ls_query = {
    "x": (153.40, 153.50),
    "y": (-27.40, -27.50),
    "time": ("2018-08-01", "2018-08-20"),
    "products": ["ga_ls5t_ard_3", "ga_ls7e_ard_3", "ga_ls8c_ard_3"],
    "measurements": [
        "nbart_green",
        "nbart_red",
        "nbart_blue",
    ],  # only get the bands we need for plotting and segmenting
    "output_crs": "EPSG:3577",
    "resolution": (-30, 30),
    "group_by": "solar_day",
}

### Load available data from Landsat satellites

In [ ]:
# Load landsat data using load_ard
ls_ds = load_ard(dc=dc, ls7_slc_off=False, **ls_query)

### Plot up some data to take a look at it
Use the `rgb` function from `dea-plotting` module to look at our area of interest in true colour

In [ ]:
rgb(ls_ds, bands=["nbart_red", "nbart_green", "nbart_blue"], col="time")

In [ ]:
image_index = 0

### Segment our image into per-pixel-polygons
Use the red band to create a polygon for each pixel. 

In [ ]:
# Input array to segment and vectorise
input_array = ls_ds.nbart_red[image_index]
input_transform = ls_ds.affine
input_crs = ls_ds.crs

# Create array with a unique value per cell
unique_pixels = np.arange(input_array.size).reshape(input_array.shape)

# Vectorise each unique feature in array
vectors = rasterio.features.shapes(
    source=unique_pixels.astype(np.int16), transform=input_transform
)

# Extract polygons and values from generator
vectors = list(vectors)
values = [value for polygon, value in vectors]
polygons = [shape(polygon) for polygon, value in vectors]

# Create a geopandas dataframe populated with the polygon shapes
ls_poly_gdf = gpd.GeoDataFrame(data={"id": values}, geometry=polygons, crs=input_crs)

### Investigate the results
Print and plot the first 5 entries in the polygon geodataframe 

In [ ]:
# print the first 5 rows in the geodataframe
ls_poly_gdf.head()

In [ ]:
# plot the first five polygons in the geodataframe, coloured by id
ls_poly_gdf.head().plot(column="id", edgecolor="black", figsize=(20, 20))
plt.show()

### Write our polygonised pixel edges for our area of interest out to file
Change the prefix in the cell below for a customised filename

In [ ]:
output_filename = create_filename(ls_ds, image_index, ls_query, prefix="NS_ls")

### Write the corresponding RGB geotiff image data out to file, for checking 

In [ ]:
# write our new segmented shapefile out to a file
ls_poly_gdf.to_file(output_filename + ".shp")

In [ ]:
# write the corresponding geotiff out to a file, for checking purposes
ls_rgb = ls_ds.isel(time=image_index).to_array()
write_cog(ls_rgb, output_filename + ".tif")

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** July 2020

**Compatible datacube version:** 

In [ ]:
print(__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)